# Configurações Iniciais para treinamento de modelo

In [15]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

import mlflow
import mlflow.sklearn
import os
import joblib
from pathlib import Path

# Defina o caminho absoluto onde quer salvar os mlruns
print(f"Mlflow path: {mlflow.get_tracking_uri()}")
mlflow_path = Path.cwd().parent / 'mlruns'
# Cria a pasta se não existir
os.makedirs(mlflow_path, exist_ok=True)

# Diz pro MLflow usar esse caminho
mlflow.set_tracking_uri(f"file:///{mlflow_path}")

print(f"MLflow tracking URI configurada para: {mlflow.get_tracking_uri()}")

RANDOM_STATE = 345

import sys
from pathlib import Path
import logging
from datetime import datetime

# Adiciona o diretório src ao path do Python
src_path = Path.cwd().parent / 'src'
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Logger customizado
from fiap import LoggerManager
log_path = Path.cwd().parent / 'logs'
LoggerManager(log_path=log_path, base_filename='model_train')

logging.info("="*50)
logging.info(f"Início do treinamento: {datetime.now()}")


2026-02-15 22:29:47,830 - INFO - Logger initialized: g:\Meu Drive\pascon_ofc\_fiap_tech_challenges\tech_challenge_fase5\logs\2026-02-15_model_train.log
2026-02-15 22:29:47,831 - INFO - ==================================================
2026-02-15 22:29:47,833 - INFO - Início do treinamento: 2026-02-15 22:29:47.833031


Mlflow path: file:///g:\Meu Drive\pascon_ofc\_fiap_tech_challenges\tech_challenge_fase5\mlruns
MLflow tracking URI configurada para: file:///g:\Meu Drive\pascon_ofc\_fiap_tech_challenges\tech_challenge_fase5\mlruns


# Carregar dados

In [16]:
logging.info("Carregar arquivo de dados processados")
df = pd.read_csv(Path.cwd().parent / 'data' / "processed_data.csv")
logging.info(f"Dados carregados: {df.shape[0]} linhas, {df.shape[1]} colunas")

logging.info("Verificar tipos e valores nulos")
logging.info(f"{df.dtypes}")
logging.info(f"Valores nulos por coluna:\n{df.isnull().sum()}")

2026-02-15 22:29:47,871 - INFO - Carregar arquivo de dados processados
2026-02-15 22:29:47,941 - INFO - Dados carregados: 1156 linhas, 21 colunas
2026-02-15 22:29:47,947 - INFO - Verificar tipos e valores nulos
2026-02-15 22:29:47,947 - INFO - fase                    int64
idade                   int64
iaa                   float64
ieg                   float64
ips                   float64
ipp                   float64
ida                   float64
mat                   float64
por                   float64
ipv                   float64
ian                   float64
defasagem               int64
genero_f                int64
genero_m                int64
instituição_tipo_1      int64
instituição_tipo_2      int64
instituição_tipo_3      int64
instituição_tipo_4      int64
instituição_tipo_5      int64
instituição_tipo_6      int64
instituição_tipo_7      int64
dtype: object
2026-02-15 22:29:47,947 - INFO - Valores nulos por coluna:
fase                  0
idade                 0
iaa  

# Separar e normalizar dados

In [17]:
X = df.drop(columns=['defasagem'])
y = df['defasagem']

# =====================================================
# Normalização
# =====================================================
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
logging.info("Dados normalizados com MinMaxScaler")
# salvar o scaler para uso futuro
joblib.dump(scaler, Path.cwd().parent / 'ml_models' / 'scaler.pkl')
logging.info(f"Scaler salvo em: {Path.cwd().parent / 'ml_models' / 'scaler.pkl'}")

2026-02-15 22:29:47,992 - INFO - Dados normalizados com MinMaxScaler
2026-02-15 22:29:48,034 - INFO - Scaler salvo em: g:\Meu Drive\pascon_ofc\_fiap_tech_challenges\tech_challenge_fase5\ml_models\scaler.pkl


In [18]:
def log_extreme_examples(y, X):
    for val in [-2, 2]:
        idx = y[y == val].index
        if not idx.empty:
            # pega a linha correta de X
            linha_X = X.iloc[idx[0]].to_dict()
            logging.info(f"Aluno com defasagem {val}: X={linha_X}, y={y.iloc[idx[0]]}")
        else:
            logging.info(f"Nenhum aluno com defasagem {val} encontrado.")

log_extreme_examples(y, X)


2026-02-15 22:29:48,080 - INFO - Aluno com defasagem -2: X={'fase': 0.0, 'idade': 10.0, 'iaa': 9.002, 'ieg': 8.6136363635, 'ips': 7.51, 'ipp': 7.1875, 'ida': 7.5, 'mat': 10.0, 'por': 5.0, 'ipv': 6.27, 'ian': 5.0, 'genero_f': 0.0, 'genero_m': 1.0, 'instituição_tipo_1': 1.0, 'instituição_tipo_2': 0.0, 'instituição_tipo_3': 0.0, 'instituição_tipo_4': 0.0, 'instituição_tipo_5': 0.0, 'instituição_tipo_6': 0.0, 'instituição_tipo_7': 0.0}, y=-2
2026-02-15 22:29:48,086 - INFO - Aluno com defasagem 2: X={'fase': 5.0, 'idade': 14.0, 'iaa': 9.168, 'ieg': 9.431818181666666, 'ips': 7.51, 'ipp': 8.125, 'ida': 8.5, 'mat': 9.0, 'por': 8.5, 'ipv': 8.674, 'ian': 10.0, 'genero_f': 0.0, 'genero_m': 1.0, 'instituição_tipo_1': 0.0, 'instituição_tipo_2': 1.0, 'instituição_tipo_3': 0.0, 'instituição_tipo_4': 0.0, 'instituição_tipo_5': 0.0, 'instituição_tipo_6': 0.0, 'instituição_tipo_7': 0.0}, y=2


# Separar em treino e teste

In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=RANDOM_STATE
)
logging.info(f"Tamanho do treino: {X_train.shape[0]} exemplos")
logging.info(f"Tamanho do teste: {X_test.shape[0]} exemplos")

2026-02-15 22:29:48,147 - INFO - Tamanho do treino: 924 exemplos
2026-02-15 22:29:48,149 - INFO - Tamanho do teste: 232 exemplos


# Treinamento do modelo

In [ ]:
# =====================================================
# Modelos e grids de hiperparâmetros
# =====================================================
modelos = {
    'Regressão Linear': LinearRegression(),
    'Árvore de Decisão': DecisionTreeRegressor(random_state=RANDOM_STATE),
    'Random Forest': RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1),
    'HistGradientBoosting': HistGradientBoostingRegressor(random_state=RANDOM_STATE)
}

param_grids = {
    'Regressão Linear': {
        'fit_intercept': [True, False],
        'positive': [True, False]
    },
    'Árvore de Decisão': {
        'max_depth': [None, 5, 10, 15],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'Random Forest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 5, 10],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    },
    'HistGradientBoosting': {
        'max_iter': [100, 200],
        'max_depth': [None, 5, 10],
        'learning_rate': [0.01, 0.1, 0.2],
        'min_samples_leaf': [20, 50]
    }
}


# =====================================================
# Treinamento com GridSearchCV + CV + MLflow + salvamento do melhor modelo
# =====================================================
logging.info("========== INÍCIO DO TREINAMENTO ==========")
resultados = {}
cv_mae = {}
melhores_estimadores = {}
melhor_r2_geral = -np.inf
melhor_modelo_geral = None
melhor_nome_geral = None
model_path = Path.cwd().parent / 'ml_models'
model_path.mkdir(parents=True, exist_ok=True)

mlflow.set_experiment(experiment_name="Defasagem Escolar - Modelos")

for nome, modelo in modelos.items():
    logging.info(f"Treinando modelo: {nome}")
    try:
        # GridSearchCV com 5-fold cross-validation e MAE
        grid = GridSearchCV(modelo, param_grids[nome], cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
        grid.fit(X_train, y_train)

        melhor_modelo = grid.best_estimator_
        melhores_estimadores[nome] = melhor_modelo
        cv_mae[nome] = -grid.best_score_

        # Predição no teste
        y_pred = melhor_modelo.predict(X_test)

        # Métricas
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)

        resultados[nome] = {'MAE': mae, 'RMSE': rmse, 'R2': r2, 'CV_MAE': cv_mae[nome]}
        logging.info(f"Modelo: {nome} | MAE: {mae:.4f} | RMSE: {rmse:.4f} | R²: {r2:.4f} | CV_MAE: {cv_mae[nome]:.4f}")

        # Log no MLflow
        with mlflow.start_run(run_name=nome):
            mlflow.log_params(grid.best_params_)
            mlflow.log_metric("MAE", mae)
            mlflow.log_metric("RMSE", rmse)
            mlflow.log_metric("R2", r2)
            mlflow.log_metric("CV_MAE", cv_mae[nome])
            mlflow.sklearn.log_model(
                sk_model=melhor_modelo,
                name="model",
                input_example=X_train[:5],  # exemplo de 5 linhas
            )

        # Verifica se é o melhor modelo geral (pelo R²)
        if r2 > melhor_r2_geral:
            melhor_r2_geral = r2
            melhor_modelo_geral = melhor_modelo
            melhor_nome_geral = nome

            # Salva localmente
            joblib.dump(melhor_modelo_geral, model_path / "best_model.pkl")
            logging.info(f"Novo melhor modelo salvo: {melhor_nome_geral} com R²={melhor_r2_geral:.4f}")

    except Exception as e:
        logging.error(f"Erro no modelo {nome}: {e}")

logging.info("========== FIM DO TREINAMENTO ==========")

# DataFrame final de resultados
df_resultados = pd.DataFrame(resultados).T.sort_values(by="R2", ascending=False)
logging.info("Ranking dos modelos por R²:\n" + df_resultados.to_string())
logging.info("===== RESULTADOS FINAIS =====")
logging.info(df_resultados.to_string())

# Informar qual foi o modelo final
logging.info(f"Modelo final selecionado automaticamente: {melhor_nome_geral}")
logging.info(melhor_modelo_geral)
logging.info(f"Modelo final salvo em: {model_path / 'best_model.pkl'}")


2026-02-15 22:29:48,191 - INFO - ========== INÍCIO DO TREINAMENTO ==========
2026-02-15 22:29:48,243 - INFO - Treinando modelo: Regressão Linear
2026-02-15 22:29:48,319 - INFO - Modelo: Regressão Linear | MAE: 0.2770 | RMSE: 0.3962 | R²: 0.7751 | CV_MAE: 0.2907


2026-02-15 22:29:56,170 - INFO - Novo melhor modelo salvo: Regressão Linear com R²=0.7751
2026-02-15 22:29:56,171 - INFO - Treinando modelo: Árvore de Decisão
2026-02-15 22:29:56,448 - INFO - Modelo: Árvore de Decisão | MAE: 0.2133 | RMSE: 0.4713 | R²: 0.6817 | CV_MAE: 0.1633


2026-02-15 22:30:03,763 - INFO - Treinando modelo: Random Forest
2026-02-15 22:30:30,340 - INFO - Modelo: Random Forest | MAE: 0.2231 | RMSE: 0.3766 | R²: 0.7968 | CV_MAE: 0.1988


2026-02-15 22:30:37,561 - INFO - Novo melhor modelo salvo: Random Forest com R²=0.7968
2026-02-15 22:30:37,577 - INFO - Treinando modelo: HistGradientBoosting
2026-02-15 22:30:46,943 - INFO - Modelo: HistGradientBoosting | MAE: 0.2308 | RMSE: 0.3934 | R²: 0.7782 | CV_MAE: 0.2159


# Validação de extremos novamente para ver se o modelo está coerente

In [ ]:
logging.info("Validando predições para alunos com defasagem extrema (-2 e 2)")
for val, idx_list in zip([-2, 2], [y[y==-2].index, y[y==2].index]):
    if len(idx_list) > 0:
        i = idx_list[0]
        logging.info(f"Aluno com defasagem {val}: X={X.iloc[i].to_dict()}, y={y.iloc[i]}")
        logging.info(f"Predição do modelo: {melhor_modelo_geral.predict([X_scaled[i]])[0]:.4f}")

2026-02-15 22:27:57,031 - INFO - Validando predições para alunos com defasagem extrema (-2 e 2)
2026-02-15 22:27:57,034 - INFO - Aluno com defasagem -2: X={'fase': 0.0, 'idade': 10.0, 'iaa': 9.002, 'ieg': 8.6136363635, 'ips': 7.51, 'ipp': 7.1875, 'ida': 7.5, 'mat': 10.0, 'por': 5.0, 'ipv': 6.27, 'ian': 5.0, 'genero_f': 0.0, 'genero_m': 1.0, 'instituição_tipo_1': 1.0, 'instituição_tipo_2': 0.0, 'instituição_tipo_3': 0.0, 'instituição_tipo_4': 0.0, 'instituição_tipo_5': 0.0, 'instituição_tipo_6': 0.0, 'instituição_tipo_7': 0.0}, y=-2
2026-02-15 22:27:57,083 - INFO - Predição do modelo: -1.5208
2026-02-15 22:27:57,084 - INFO - Aluno com defasagem 2: X={'fase': 5.0, 'idade': 14.0, 'iaa': 9.168, 'ieg': 9.431818181666666, 'ips': 7.51, 'ipp': 8.125, 'ida': 8.5, 'mat': 9.0, 'por': 8.5, 'ipv': 8.674, 'ian': 10.0, 'genero_f': 0.0, 'genero_m': 1.0, 'instituição_tipo_1': 0.0, 'instituição_tipo_2': 1.0, 'instituição_tipo_3': 0.0, 'instituição_tipo_4': 0.0, 'instituição_tipo_5': 0.0, 'instituição_ti